In [1]:
import cv2
import numpy as np
import os
import pandas as pd
from PIL import Image
import imagehash

In [2]:
%load_ext memory_profiler

In [4]:
def framing(video_path):

    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(5))
    print("fps:", fps)
    if(fps==0):
        print("Not available")
        return 0

    og_fr = []
    grayed_fr = []
    while True:
        ret, frame = cap.read()
        if ret:
            resized_frame = cv2.resize(frame, (224, 224), interpolation= cv2.INTER_LINEAR)

            grayed = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            og_fr.append(frame)
            grayed_fr.append(grayed)
        else:
            break
            
    return grayed_fr, og_fr

In [17]:
def keyframeDetection(video_path):
    grayed_frames, og_frames = framing(video_path)
    
    if(len(grayed_frames)==0):
        print("Cannot compare video because of zero fps.")
        return 0
    
    else:
        video_name = (video_path.split('/')[-1]).split('.')[0]
        keyframePath = 'keyframes/'+video_name
        if not os.path.exists(keyframePath):
            os.makedirs(keyframePath)
        
        diff = []
        for i in range(0,len(grayed_frames)-1):
            diff.append(cv2.absdiff(grayed_frames[i], grayed_frames[i+1]))

        mn_all_frames = np.mean(diff)         #This gives mean
        st_d_all_frames = np.std(diff)        #This gives standard deviation
        a = 8                                 #Setting a random value we can modify it to any value 
        ts = mn_all_frames + (a * st_d_all_frames)   # global threshold
        
        local_ts = []
        for i in range(len(diff)):
            mn = np.mean(diff[i])             #Calculating the mean for each frame
            st_d = np.std(diff[i])            #Calculating the standard deviation for each frame
            fr_ts = mn + (5*st_d)             #Finding the threshold values for each frame
            local_ts.append([i,fr_ts])
            

        frame_num = []
        count = 0
        for i,l_tr in(local_ts):
            if l_tr >= ts:                   #Comparing the threshold values to the standard threshold/global threshold values
                cv2.imwrite(keyframePath+'/'+str(i)+'.jpg',grayed_frames[i]) 
                frame_num.append(i)
                count+=1

        print("Total Number of keyframes saved: {}".format(count))    
        return frame_num, keyframePath

In [18]:
def imageDist(img1, img2):
    img1_hash = imagehash.phash(Image.open(img1))
    img2_hash = imagehash.phash(Image.open(img2))
    dist = img1_hash - img2_hash
    return int(dist)

In [19]:
def videoSimilarity(video1_path, video2_path):
    v1_keyframe_num, keyframe1_path = keyframeDetection(video1_path)
    v2_keyframe_num, keyframe2_path = keyframeDetection(video2_path)
    
    dist_mat = []
    count=0
    for i in v1_keyframe_num:
        row = []
        p1 = keyframe1_path+'/'+str(i)+'.jpg'
        for j in v2_keyframe_num:  
            p2 = keyframe2_path+'/'+str(j)+'.jpg'
            dist = imageDist(p1, p2)
            row.append(dist)

        count = count+1
        print("row",count, end='\r')
        dist_mat.append(row)
        
    sim_mat = 100-np.array(dist_mat)
    sim_df = pd.DataFrame(sim_mat)   
    sim_percent = ((sim_df.max(axis=1).sum())/len(sim_df))
    return sim_percent

In [20]:
videoSimilarity("index-videos/test3.mp4", "index-videos/test4.mp4")

fps: 29
Total Number of keyframes saved: 11
fps: 30
Total Number of keyframes saved: 9


85.27272727272727

In [10]:
videoSimilarity("index-videos/test5.mp4", "index-videos/test6.mp4")

fps: 30
Total Number of keyframes saved: 34
fps: 30
Total Number of keyframes saved: 14


78.47058823529412

In [11]:
videoSimilarity("index-videos/test7.mp4", "index-videos/test8.mp4")

fps: 30
Total Number of keyframes saved: 74
fps: 25
Total Number of keyframes saved: 53


97.72972972972973

In [15]:
y = 100-pd.DataFrame(x)
sim_percent = ((y.max(axis=1).sum())/len(y))
sim_percent

96.36363636363636

In [35]:
y

,0,1,2,3,4,5,6,7,8
0,96,94,92,92,86,86,86,86,90
1,96,96,94,94,90,90,90,88,90
2,96,98,100,100,90,90,90,90,94
3,90,90,90,90,96,96,96,94,90
4,90,90,90,90,96,96,96,94,90
5,92,92,92,92,96,96,96,94,90
6,92,92,92,92,96,96,96,94,90
7,90,90,92,92,96,96,96,96,88
8,90,90,92,92,96,96,96,96,88
9,90,90,92,92,96,96,96,96,88


In [18]:
x

[[4, 6, 8, 8, 14, 14, 14, 14, 10],
 [4, 4, 6, 6, 10, 10, 10, 12, 10],
 [4, 2, 0, 0, 10, 10, 10, 10, 6],
 [10, 10, 10, 10, 4, 4, 4, 6, 10],
 [10, 10, 10, 10, 4, 4, 4, 6, 10],
 [8, 8, 8, 8, 4, 4, 4, 6, 10],
 [8, 8, 8, 8, 4, 4, 4, 6, 10],
 [10, 10, 8, 8, 4, 4, 4, 4, 12],
 [10, 10, 8, 8, 4, 4, 4, 4, 12],
 [10, 10, 8, 8, 4, 4, 4, 4, 12],
 [10, 10, 8, 8, 4, 4, 4, 4, 12]]

In [22]:
i = Image.open("keyframes\speeder-bmw/0.jpg")
h_value = imagehash.phash(i)
print(h_value)

f6a4294b427a6d17


In [25]:
print(len(h_value))
print(type(h_value))

64
<class 'imagehash.ImageHash'>
